<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Описание данных:**

**Признаки**: пол, возраст и зарплата застрахованного, количество членов его семьи.

**Целевой признак**: количество страховых выплат клиенту за последние 5 лет.


## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import warnings
from IPython.display import display
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
df= pd.read_csv('/datasets/insurance.csv')

In [2]:
display(df.head())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


5000 строк и 5 колонок. Пропусков нет.

Проверим есть ли какие то аномальные значения в данных.

In [4]:
df['Пол'].unique()

array([1, 0])

2 пола.

In [5]:
print('Минимальный возраст',df['Возраст'].min())
print('Максимальный возраст',df['Возраст'].max())

Минимальный возраст 18.0
Максимальный возраст 65.0


In [6]:
df['Возраст'].unique()

array([41., 46., 29., 21., 28., 43., 39., 25., 36., 32., 38., 23., 40.,
       34., 26., 42., 27., 33., 47., 30., 19., 31., 22., 20., 24., 18.,
       37., 48., 45., 44., 52., 49., 35., 56., 65., 55., 57., 54., 50.,
       53., 51., 58., 59., 60., 61., 62.])

Возраст от 18 до 65, дробных значений нет. 

In [7]:
df['Члены семьи'].unique()

array([1, 0, 2, 4, 3, 5, 6])

От 0 до 6 членов семьи. Правдоподобные цифры. 

In [8]:
print('Минимальная зарплата',df['Зарплата'].min())
print('Максимальная зарплата',df['Зарплата'].max())

Минимальная зарплата 5300.0
Максимальная зарплата 79000.0


Зарплата в переделах от 5300 и до 79000. 5300 ниже МРОТ, но это может быть зарплата человека, работающего на частичной занятости. 

In [9]:
df['Страховые выплаты'].unique()

array([0, 1, 2, 3, 5, 4])

Максимально было 5 страховых выплат за 5 лет. Вполне может быть. 

Проверим наличие дубликатов в данных

In [10]:
print('Количество дубликатов:', df.duplicated().sum())

Количество дубликатов: 153


Посмотрим какой это процент от общих данных. 

In [11]:
print('Доля дубликатов:', df.duplicated().sum()/len(df))


Доля дубликатов: 0.0306


Дубликатов всего 0.03%. Это очень маленький процент. Удалим дуликаты, чтобы улучшить обучаемость модели.

In [12]:
df= df.drop_duplicates()
df.shape

(4847, 5)

Данные загружены, аномальных значений нет, дубликаты удалены.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.


Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос**: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится. 


**Обоснование:** 

Посмотрим как должен выглядеть расчет.

Подставим в формулу $ a = X_1w $  расчет w $ w = (X^T X)^{-1} X^T y $

$$a=X_1((X^T X)^{-1} X^T y) $$


Раскроем скобки, используя следующее свойство обратной матрицы:$(AB)^{-1} = B^{-1} A^{-1}$.  

$$  a = X_1X^{-1}(X^T)^{-1}X^T y $$


Умножение матрицы $ X^T$ на обратную матрицу$(X^T)^{-1}$ равно единичной матрице **E**:

$$  a = X_1X^{-1}E y = X_1X^{-1}y$$

Теперь умножим матрицу на обратимую матрицу **P**:

$$a_1 = (X_1P)((XP)^T XP)^{-1}(XP)^Ty$$

Раскроем скобки, используя свойство обратной матрицы: $(AB)^{-1} = B^{-1} A^{-1}$ и  свойство транспорнированной матрицы:$(AB)^T = B^T A^T$ :

$$a_1 = (X_1P)(XP)^{-1}((XP)^T)^{-1}(XP)^T y =(X_1P)P^{-1}X^{-1}(X^T)^{-1}(P^T)^{-1}P^T X^T y=X_1PP^{-1}X^{-1}(X^T)^{-1}(P^T)^{-1}P^T X^T y$$

Результатом умножения матрицы на обратную матрицу будет единичная мартица **E**: $PP^{-1}=E$,  $(P^T)^{-1}P^T=E$,  $(X^T)^{-1}X^T=E$:

$$a_1=X_1EX^{-1}(X^T)^{-1}E X^T y= X_1X^{-1}(X^T)^{-1} X^T y=X_1X^{-1}Ey= X_1X^{-1}y$$

Получается $a=a_1$

## Алгоритм преобразования

**Алгоритм**
Для защиты информации умножим матрицу признаков на обратимую матрицу, которая генерируется случайным образом.

Этапы алгоритма:

- Создание случайной матрицы размерностью **n**, где **n**- ширина матрицы признаков.
- Проверка матрицы на обратимость.
- Умножение матрицы признаков на случайную обратимую матрицу 
- Применение модели на преобразованных признаках 


**Обоснование**

Матрица признаков имеет размерность **m** x **n**, где **n**- ширина матрицы, те количество признаков для регрессии. Случайная матрица должна быть квадратной размера **n**x**n**, тк обратимая матрица может быть только квадратной

## Проверка алгоритма

Разделим данные на признаки и целевой признак

In [13]:
features= df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

Проверим размерность

In [14]:
features.shape

(4847, 4)

In [15]:
target.shape

(4847,)

В таблице признаков 4 признака. Значит необходимо создать случайную матрицу размера 4х4 

In [16]:
temp= np.random.normal(size =(4,4))
print(temp)

[[-1.46343691  1.12514174  0.55904594 -0.97676825]
 [ 0.29750172 -0.01957581  0.98370264  0.22439967]
 [-2.4353402  -0.6005383  -1.42931028 -1.11041002]
 [ 1.21514794 -1.84270487 -0.238664    0.47662136]]


Проверим является ли она обратимой

In [17]:
invert = np.linalg.inv(temp)
print(invert)

[[ 0.88783095 -1.3523955  -0.71535696  0.78960509]
 [-0.02690443 -0.4984896  -0.2761547  -0.46381355]
 [ 0.24324986  0.95919179  0.03442867  0.12711632]
 [-2.24574051  2.00099286  0.77338079 -1.64453477]]


Матрица обратимая

Умножим матрицу с признаками на случайную обратимую матрицу

In [18]:
features_encr=features@temp
features_encr.shape

(4847, 4)

Напишем функцию для обучения модели и расчета R2

In [19]:
def r2_score(features, target):
    model = LinearRegression()
    model.fit(features, target)
    r2 = model.score(features, target)
    return r2

Посчитаем R2 на исходных данных

In [20]:
r2_score(features, target)

0.4302010044852067

Теперь посчитаем R2 на преобразованных данных

In [21]:
r2_score(features_encr, target)

0.43020100448520027

Метрика R2 не изменилась. Колебания начинаются с 16 знака после запятой, что можно считать очень незначительным изменением.

**Вывод**. Качество линейной регрессии не изменилось при использовании преобразованных данных. Метрика R2 показала одинаковые значения как на исходных данных, так и на преобразованных. 

Данные можно зашифровать от прочтения\распознавания, если подобрать правильную матрицу. 